# Task 2: Data Insights

In [170]:
import pandas as pd
import numpy as np
import datetime as DT
from datetime import timedelta
import io
from sklearn.impute import SimpleImputer

In [171]:
# First rows of the sheets are comments, not part of dataframes
df_transactions = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='Transactions', skiprows=1)
df_new_customer_list = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='NewCustomerList', skiprows=1)
df_customer_demographic = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerDemographic', skiprows=1)
df_customer_address = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerAddress', skiprows=1)

In [172]:
# Make copies for fail safe.
df_transactions_copy = df_transactions
df_new_customer_list_copy = df_new_customer_list
df_customer_demographic_copy = df_customer_demographic
df_customer_address_copy = df_customer_address

In [173]:
df_transactions.sample()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
6135,6136,46,3375,2017-08-25,1.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0


In [174]:
df_new_customer_list.sample()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,1.01,1.01,1.01,1.01,1,1,1.71875


In [175]:
df_customer_demographic.sample()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
2731,2732,Clarine,Yushkin,Female,36,1961-03-01,Help Desk Operator,Health,Mass Customer,N,ð¾ ð ð ð ð ð ð ð§,No,7.0


In [176]:
df_customer_address.sample()

,customer_id,address,postcode,state,country,property_valuation
1286,1291,8 Kinsman Alley,4304,QLD,Australia,2


## Analyze and Clean New Customer List

In [177]:
df_new_customer_list.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,QLD,Australia,6,0.56,0.7000,0.8750,0.743750,1,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,NSW,Australia,11,0.89,0.8900,1.1125,0.945625,1,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,1.01,1.0100,1.0100,1.010000,1,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,QLD,Australia,1,0.87,1.0875,1.0875,1.087500,4,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,NSW,Australia,9,0.52,0.5200,0.6500,0.650000,4,4,1.703125


In [178]:
df_new_customer_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   first_name                           1000 non-null   object        
 1   last_name                            971 non-null    object        
 2   gender                               1000 non-null   object        
 3   past_3_years_bike_related_purchases  1000 non-null   int64         
 4   DOB                                  983 non-null    datetime64[ns]
 5   job_title                            894 non-null    object        
 6   job_industry_category                835 non-null    object        
 7   wealth_segment                       1000 non-null   object        
 8   deceased_indicator                   1000 non-null   object        
 9   owns_car                             1000 non-null   object        
 10  tenure       

We have five unnamed columns. We are not sure what they represents, so it is better to consult with the client or drop them for our analysis.

In [179]:
df_new_customer_list.drop(['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20'], axis = 1, inplace=True)
df_new_customer_list.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125


We should convert DOB to age.

In [180]:
df_transactions['transaction_date'].sort_values()

516     2017-01-01
5876    2017-01-01
3459    2017-01-01
12484   2017-01-01
19130   2017-01-01
           ...    
605     2017-12-30
15269   2017-12-30
19906   2017-12-30
15756   2017-12-30
12003   2017-12-30
Name: transaction_date, Length: 20000, dtype: datetime64[ns]

We see that the data is collected in 2017. We should convert age to 2017.

In [181]:
# https://stackoverflow.com/questions/26788854/pandas-get-the-age-from-a-date-example-date-of-birth
# https://stackoverflow.com/questions/58948809/why-do-i-get-valueerror-nattype-does-not-support-strftime-even-though-its-no

df_new_customer_list['DOB'] = pd.to_datetime(df_new_customer_list['DOB'], errors='coerce', format='%Y-%m-%d')

year_2017 = pd.Timestamp('2017-12-31')

df_new_customer_list['age'] = (year_2017 - df_new_customer_list['DOB']).astype('<m8[Y]')

# drop DOB
df_new_customer_list.drop(['DOB'], axis = 1, inplace=True)

df_new_customer_list.head()


,first_name,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value,age
0,Chickie,Brister,Male,86,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750,60.0
1,Morly,Genery,Male,69,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750,47.0
2,Ardelis,Forrester,Female,10,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750,43.0
3,Lucine,Stutt,Female,64,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125,38.0
4,Melinda,Hadlee,Female,34,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125,52.0


## RFM

In [182]:
df_transactions.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

No key variables are missing

In [183]:
# https://towardsdatascience.com/recency-frequency-monetary-model-with-python-and-how-sephora-uses-it-to-optimize-their-google-d6a0707c5f17

snapshot_date = df_transactions['transaction_date'].max() + timedelta(days=1)
print(snapshot_date)


data_process = df_transactions.groupby(['customer_id']).agg({
        'transaction_date': lambda x: (snapshot_date - x.max()).days,
        'transaction_id': 'count',
        'list_price': 'sum'})

data_process.rename(columns={'transaction_date': 'Recency',
                         'transaction_id': 'Frequency',
                         'list_price': 'MonetaryValue'}, inplace=True)

data_process.head()


2017-12-31 00:00:00


,Recency,Frequency,MonetaryValue
customer_id,,,
1,8,11,9084.45
2,129,3,4149.07
3,103,8,9888.23
4,196,2,1047.72
5,17,6,5903.20


In [184]:
# --Calculate R and F groups--
# Create labels for Recency and Frequency
r_labels = range(4, 0, -1); f_labels = range(1, 5)
# Assign these labels to 4 equal percentile groups 
r_groups = pd.qcut(data_process['Recency'], q=4, labels=r_labels)
# Assign these labels to 4 equal percentile groups 
f_groups = pd.qcut(data_process['Frequency'], q=4, labels=f_labels)
# Create new columns R and F 
data_process = data_process.assign(R = r_groups.values, F = f_groups.values)
data_process.head()

,Recency,Frequency,MonetaryValue,R,F
customer_id,,,,,
1,8,11,9084.45,4,4
2,129,3,4149.07,1,1
3,103,8,9888.23,1,4
4,196,2,1047.72,1,1
5,17,6,5903.20,4,2


In [185]:
# Create labels for MonetaryValue
m_labels = range(1, 5)
# Assign these labels to three equal percentile groups 
m_groups = pd.qcut(data_process['MonetaryValue'], q=4, labels=m_labels)
# Create new column M
data_process = data_process.assign(M = m_groups.values)

In [186]:
# Calculate RFM_Score
data_process['RFM_Score'] = data_process[['R','F','M']].sum(axis=1)
data_process

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Score
customer_id,,,,,,,
1,8,11,9084.45,4,4,4,12.0
2,129,3,4149.07,1,1,1,3.0
3,103,8,9888.23,1,4,4,9.0
4,196,2,1047.72,1,1,1,3.0
5,17,6,5903.20,4,2,2,8.0
...,...,...,...,...,...,...,...
3497,53,3,3744.07,2,1,1,4.0
3498,128,6,5177.06,1,2,2,5.0
3499,52,7,7673.48,2,3,3,8.0


In [187]:
data_process.describe()

,Recency,Frequency,MonetaryValue,RFM_Score
count,3494.000000,3494.000000,3494.000000,3494.000000
mean,61.813681,5.724098,6341.324837,7.253578
std,57.787120,2.325873,2959.132004,2.651282
min,1.000000,1.000000,60.340000,3.000000
25%,18.000000,4.000000,4198.390000,5.000000
50%,45.000000,6.000000,6026.800000,7.000000
75%,86.000000,7.000000,8208.837500,9.000000
max,354.000000,14.000000,19071.320000,12.000000


We can set RFM_Score higher than 9 to be Tier 4, the second quantile to be Tier 3, and so on. The higher the tier, the better value the customer.

In [188]:
for index, row in data_process.iterrows():
    if row['RFM_Score'] >= 9:
        data_process.at[index, 'Tier'] = 4
    elif row['RFM_Score'] >= 7:
        data_process.at[index, 'Tier'] = 3
    elif row['RFM_Score'] >= 5:
        data_process.at[index, 'Tier'] = 2
    else:
        data_process.at[index, 'Tier'] = 1
        
data_process.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Score,Tier
customer_id,,,,,,,,
1,8,11,9084.45,4,4,4,12.0,4.0
2,129,3,4149.07,1,1,1,3.0,1.0
3,103,8,9888.23,1,4,4,9.0,4.0
4,196,2,1047.72,1,1,1,3.0,1.0
5,17,6,5903.20,4,2,2,8.0,3.0


In [189]:
data_process.reset_index(level=0, inplace=True)
rfm = data_process[['customer_id', 'Tier']]
rfm.head()

,customer_id,Tier
0,1,4.0
1,2,1.0
2,3,4.0
3,4,1.0
4,5,3.0


## Create Old Customer List as Training Data

In [190]:
df_old_customer_list = df_customer_demographic
df_old_customer_list = df_old_customer_list.merge(df_customer_address, how='left', on='customer_id')
df_old_customer_list.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0,060 Morning Avenue,2016.0,New South Wales,Australia,10.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0,6 Meadow Vale Court,2153.0,New South Wales,Australia,10.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0,NaN,NaN,NaN,NaN,NaN
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0,0 Holy Cross Court,4211.0,QLD,Australia,9.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0,17979 Del Mar Point,2448.0,New South Wales,Australia,4.0


Merge number of transactions to the old customer list

In [191]:
df_old_customer_list = df_old_customer_list.merge(rfm, how='left', on='customer_id')
df_old_customer_list.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,address,postcode,state,country,property_valuation,Tier
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0,060 Morning Avenue,2016.0,New South Wales,Australia,10.0,4.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0,6 Meadow Vale Court,2153.0,New South Wales,Australia,10.0,1.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0,NaN,NaN,NaN,NaN,NaN,4.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0,0 Holy Cross Court,4211.0,QLD,Australia,9.0,1.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0,17979 Del Mar Point,2448.0,New South Wales,Australia,4.0,3.0


In [192]:
df_old_customer_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 19 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4000 non-null   int64         
 1   first_name                           4000 non-null   object        
 2   last_name                            3875 non-null   object        
 3   gender                               4000 non-null   object        
 4   past_3_years_bike_related_purchases  4000 non-null   int64         
 5   DOB                                  3913 non-null   datetime64[ns]
 6   job_title                            3494 non-null   object        
 7   job_industry_category                3344 non-null   object        
 8   wealth_segment                       4000 non-null   object        
 9   deceased_indicator                   4000 non-null   object        
 10  default     

Convert DOB to age with respect to year 2017

In [193]:
df_old_customer_list['DOB'] = pd.to_datetime(df_old_customer_list['DOB'], errors='coerce', format='%Y-%m-%d')

year_2017 = pd.Timestamp('2017-12-31')

df_old_customer_list['age'] = (year_2017 - df_old_customer_list['DOB']).astype('<m8[Y]')

# drop DOB
df_old_customer_list.drop(['DOB'], axis = 1, inplace=True)

df_old_customer_list.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,address,postcode,state,country,property_valuation,Tier,age
0,1,Laraine,Medendorp,F,93,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0,060 Morning Avenue,2016.0,New South Wales,Australia,10.0,4.0,64.0
1,2,Eli,Bockman,Male,81,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0,6 Meadow Vale Court,2153.0,New South Wales,Australia,10.0,1.0,37.0
2,3,Arlin,Dearle,Male,61,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0,NaN,NaN,NaN,NaN,NaN,4.0,63.0
3,4,Talbot,NaN,Male,33,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0,0 Holy Cross Court,4211.0,QLD,Australia,9.0,1.0,56.0
4,5,Sheila-kathryn,Calton,Female,56,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0,17979 Del Mar Point,2448.0,New South Wales,Australia,4.0,3.0,40.0


## Data Cleaning for Both Old and New Customer List

### Drop Unecessary Columns

In [194]:
old_columns = df_old_customer_list.columns
new_columns = df_new_customer_list.columns

for col in old_columns:
    if col not in new_columns:
        print(col + ' in df_old_customer_list not exist in df_new_customer_list')
        
print()

for col in new_columns:
    if col not in old_columns:
        print(col + ' in df_new_customer_list not exist in df_old_customer_list')

customer_id in df_old_customer_list not exist in df_new_customer_list
default in df_old_customer_list not exist in df_new_customer_list
Tier in df_old_customer_list not exist in df_new_customer_list

Rank in df_new_customer_list not exist in df_old_customer_list
Value in df_new_customer_list not exist in df_old_customer_list


We should drop columns except customer_id and Tier. customer_id will be excluded from our analysis. Tier will be the y value for machine learning models.

In [195]:
df_old_customer_list.drop('default', axis=1, inplace=True)
df_new_customer_list.drop(['Rank', 'Value'], axis=1, inplace=True)

In [196]:
df_old_customer_list.head(2)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Tier,age
0,1,Laraine,Medendorp,F,93,Executive Secretary,Health,Mass Customer,N,Yes,11.0,060 Morning Avenue,2016.0,New South Wales,Australia,10.0,4.0,64.0
1,2,Eli,Bockman,Male,81,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,6 Meadow Vale Court,2153.0,New South Wales,Australia,10.0,1.0,37.0


In [197]:
df_new_customer_list.head(2)

,first_name,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,age
0,Chickie,Brister,Male,86,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,60.0
1,Morly,Genery,Male,69,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,47.0


Drop categorical variables with more than 15 values.

In [198]:
len(df_old_customer_list['job_title'].unique())

196

`job_title` for both df need to be dropped

In [199]:
len(df_old_customer_list['job_industry_category'].unique())

10

In [200]:
len(df_new_customer_list['job_industry_category'].unique())

10

In [201]:
len(df_old_customer_list['wealth_segment'].unique())

3

In [202]:
len(df_new_customer_list['wealth_segment'].unique())

3

In [203]:
len(df_old_customer_list['address'].unique())

3994

`address` for both df need to be dropped

`postcode` need to be dropped too since `state` is enough as a address variable

In [204]:
df_old_customer_list.drop(['job_title', 'address', 'postcode'], axis=1, inplace=True)
df_new_customer_list.drop(['job_title', 'address', 'postcode'], axis=1, inplace=True)

In [205]:
df_old_customer_list.sample(1)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,state,country,property_valuation,Tier,age
1491,1492,Cordey,Tomik,Female,51,NaN,Mass Customer,N,No,7.0,NSW,Australia,3.0,4.0,37.0


In [206]:
df_new_customer_list.sample(1)

,first_name,last_name,gender,past_3_years_bike_related_purchases,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,state,country,property_valuation,age
788,Eleonora,Wiszniewski,Female,49,Financial Services,Affluent Customer,N,No,9,QLD,Australia,2,37.0


In [221]:
df_old_customer_list['country'].unique()

array(['Australia', nan], dtype=object)

In [222]:
df_new_customer_list['country'].unique()

array(['Australia'], dtype=object)

All data should be collected in Australia. We can remove the `country` column

In [223]:
df_old_customer_list.drop(['country'], axis=1, inplace=True)
df_new_customer_list.drop(['country'], axis=1, inplace=True)

For `deceased_indicator`, we should remove the customer if deceased, then drop column.

In [244]:
df_old_customer_list['deceased_indicator'].value_counts()

N    3998
Y       2
Name: deceased_indicator, dtype: int64

In [245]:
df_old_customer_list.drop(df_old_customer_list[df_old_customer_list['deceased_indicator'] == 'Y'].index, inplace=True)
df_old_customer_list['deceased_indicator'].value_counts()

N    3998
Name: deceased_indicator, dtype: int64

In [246]:
df_new_customer_list['deceased_indicator'].value_counts()

N    1000
Name: deceased_indicator, dtype: int64

In [247]:
df_old_customer_list.drop(['deceased_indicator'], axis=1, inplace=True)
df_new_customer_list.drop(['deceased_indicator'], axis=1, inplace=True)

### Feature Engineering

In [207]:
df_old_customer_list.isnull().sum()

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                  87
state                                    4
country                                  4
property_valuation                       4
Tier                                   507
age                                     87
dtype: int64

For the missing Tier, that means 507 customers do not have transaction data in 2017. It is either caused by incomplete data, or they did not make any purchases. For now, we assume the transaction data is complete and make them Tier 0, for no purchase made.

In [210]:
values = {'Tier': 0}
df_old_customer_list.fillna(value=values, inplace=True)
df_old_customer_list.isnull().sum()

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                  87
state                                    4
country                                  4
property_valuation                       4
Tier                                     0
age                                     87
dtype: int64

In [211]:
df_old_customer_list.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,state,country,property_valuation,Tier,age
0,1,Laraine,Medendorp,F,93,Health,Mass Customer,N,Yes,11.0,New South Wales,Australia,10.0,4.0,64.0
1,2,Eli,Bockman,Male,81,Financial Services,Mass Customer,N,Yes,16.0,New South Wales,Australia,10.0,1.0,37.0
2,3,Arlin,Dearle,Male,61,Property,Mass Customer,N,Yes,15.0,NaN,NaN,NaN,4.0,63.0
3,4,Talbot,NaN,Male,33,IT,Mass Customer,N,No,7.0,QLD,Australia,9.0,1.0,56.0
4,5,Sheila-kathryn,Calton,Female,56,NaN,Affluent Customer,N,Yes,8.0,New South Wales,Australia,4.0,3.0,40.0


Analye job_industry_category

In [212]:
df_old_customer_list['job_industry_category'].unique()

array(['Health', 'Financial Services', 'Property', 'IT', nan, 'Retail',
       'Argiculture', 'Manufacturing', 'Telecommunications',
       'Entertainment'], dtype=object)

In [213]:
df_new_customer_list['job_industry_category'].unique()

array(['Manufacturing', 'Property', 'Financial Services', 'Entertainment',
       'Retail', 'IT', 'Telecommunications', 'Health', nan, 'Argiculture'],
      dtype=object)

The job categories are quite consistent. We can replace nan values with 'Unemployed'

In [214]:
values = {'job_industry_category': 'Unemployed'}
df_old_customer_list.fillna(value=values, inplace=True)
df_old_customer_list.isnull().sum()

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
job_industry_category                    0
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                  87
state                                    4
country                                  4
property_valuation                       4
Tier                                     0
age                                     87
dtype: int64

In [215]:
df_new_customer_list.fillna(value=values, inplace=True)
df_new_customer_list.isnull().sum()

first_name                              0
last_name                              29
gender                                  0
past_3_years_bike_related_purchases     0
job_industry_category                   0
wealth_segment                          0
deceased_indicator                      0
owns_car                                0
tenure                                  0
state                                   0
country                                 0
property_valuation                      0
age                                    17
dtype: int64

`tenure` and `age` missing values can be filled with mean values.

In [217]:
df_old_customer_list.fillna(df_old_customer_list.mean(), inplace=True)
df_old_customer_list.isnull().sum()

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
job_industry_category                    0
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
state                                    4
country                                  4
property_valuation                       0
Tier                                     0
age                                      0
dtype: int64

In [218]:
df_new_customer_list.fillna(df_new_customer_list.mean(), inplace=True)
df_new_customer_list.isnull().sum()

first_name                              0
last_name                              29
gender                                  0
past_3_years_bike_related_purchases     0
job_industry_category                   0
wealth_segment                          0
deceased_indicator                      0
owns_car                                0
tenure                                  0
state                                   0
country                                 0
property_valuation                      0
age                                     0
dtype: int64

Missing state

In [224]:
df_old_customer_list[df_old_customer_list['state'].isnull()]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,state,property_valuation,Tier,age
2,3,Arlin,Dearle,Male,61,Property,Mass Customer,N,Yes,15.0,NaN,7.514014,4.0,63.0
9,10,Fiorenze,Birdall,Female,49,Financial Services,Mass Customer,N,Yes,20.0,NaN,7.514014,3.0,29.0
21,22,Deeanne,Durtnell,Female,79,IT,Mass Customer,N,No,11.0,NaN,7.514014,4.0,55.0
22,23,Olav,Polak,Male,43,Unemployed,High Net Worth,N,Yes,1.0,NaN,7.514014,4.0,22.0


In [228]:
df_old_customer_list['state'].value_counts()

NSW                2052
VIC                 939
QLD                 837
New South Wales      86
Victoria             82
Name: state, dtype: int64

Value of NSW is far more than VIC and QLD, even after consistency adjustment. we can replace nan with NSW.

In [229]:
values = {'state': 'NSW'}
df_old_customer_list.fillna(value=values, inplace=True)
df_old_customer_list.isnull().sum()

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
job_industry_category                    0
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
state                                    0
property_valuation                       0
Tier                                     0
age                                      0
dtype: int64

In [230]:
df_new_customer_list.isnull().sum()

first_name                              0
last_name                              29
gender                                  0
past_3_years_bike_related_purchases     0
job_industry_category                   0
wealth_segment                          0
deceased_indicator                      0
owns_car                                0
tenure                                  0
state                                   0
property_valuation                      0
age                                     0
dtype: int64

### Consistency Check

In [249]:
df_old_customer_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3998 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          3998 non-null   int64  
 1   first_name                           3998 non-null   object 
 2   last_name                            3873 non-null   object 
 3   gender                               3998 non-null   object 
 4   past_3_years_bike_related_purchases  3998 non-null   int64  
 5   job_industry_category                3998 non-null   object 
 6   wealth_segment                       3998 non-null   object 
 7   owns_car                             3998 non-null   object 
 8   tenure                               3998 non-null   float64
 9   state                                3998 non-null   object 
 10  property_valuation                   3998 non-null   float64
 11  Tier                          

In [248]:
df_new_customer_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   first_name                           1000 non-null   object 
 1   last_name                            971 non-null    object 
 2   gender                               1000 non-null   object 
 3   past_3_years_bike_related_purchases  1000 non-null   int64  
 4   job_industry_category                1000 non-null   object 
 5   wealth_segment                       1000 non-null   object 
 6   owns_car                             1000 non-null   object 
 7   tenure                               1000 non-null   int64  
 8   state                                1000 non-null   object 
 9   property_valuation                   1000 non-null   int64  
 10  age                                  1000 non-null   float64
dtypes: float64(1), int64(3), object

#### Gender

In [231]:
df_old_customer_list['gender'].unique()

array(['F', 'Male', 'Female', 'U', 'Femal', 'M'], dtype=object)

In [232]:
df_new_customer_list['gender'].unique()

array(['Male', 'Female', 'U'], dtype=object)

We should make gender consistent with 'F', 'M', and 'U'

In [233]:
values = {'Male': 'M', 'Female': 'F', 'Femal': 'F'}
df_old_customer_list.replace(to_replace=values, inplace=True)
df_old_customer_list['gender'].unique()

array(['F', 'M', 'U'], dtype=object)

In [234]:
df_new_customer_list.replace(to_replace=values, inplace=True)
df_new_customer_list['gender'].unique()

array(['M', 'F', 'U'], dtype=object)

#### Wealth Segment

In [238]:
df_old_customer_list['wealth_segment'].unique()

array(['Mass Customer', 'Affluent Customer', 'High Net Worth'],
      dtype=object)

In [239]:
df_new_customer_list['wealth_segment'].unique()

array(['Mass Customer', 'Affluent Customer', 'High Net Worth'],
      dtype=object)

#### Owns Car

In [250]:
df_old_customer_list['owns_car'].unique()

array(['Yes', 'No'], dtype=object)

In [251]:
df_new_customer_list['owns_car'].unique()

array(['Yes', 'No'], dtype=object)

#### State

In [252]:
df_old_customer_list['state'].unique()

array(['New South Wales', 'NSW', 'QLD', 'VIC', 'Victoria'], dtype=object)

In [253]:
df_new_customer_list['state'].unique()

array(['QLD', 'NSW', 'VIC'], dtype=object)

We should make state consistent with "NSW", "QLD", and "VIC"

In [254]:
values = {'New South Wales': 'NSW', 'Victoria': 'VIC'}
df_old_customer_list.replace(to_replace=values, inplace=True)
df_old_customer_list['state'].unique()

array(['NSW', 'QLD', 'VIC'], dtype=object)